In [4]:
import psycopg2
import geopandas as gpd
import pandas as pd
from jinja2 import Environment, FileSystemLoader
from config import config

# CONEXIÓN a la BASE DE DATOS

params = config()
conn = psycopg2.connect(**params)

comarca = 'vera'
zona = 'losar'

# CONDICIONES SINÓPTICAS

sql = '''WITH inc as 
        (SELECT a.*, b.descripcion_corta as causa
        FROM 
            incendios_proximos_ze a,
            causas b
        WHERE 
            a.idcausa = b.id
            and b.ididioma = 0
        ORDER by tot desc)
        SELECT inc.*, c.texto, c.descripcion
        FROM 
            inc,
            texto_situacion_sinoptica c
        WHERE
            c.idpif = inc.idpif
        '''

incendios_cs = gpd.read_postgis(sql, conn)
incendios_cs['epa'] = (pd.DatetimeIndex(incendios_cs.deteccion).month > 5) & (pd.DatetimeIndex(incendios_cs.deteccion).month < 11)
ss = incendios_cs[['idpif', 'descripcion']].groupby(['descripcion']).count()
ss_epa = incendios_cs[incendios_cs.epa == True][['idpif', 'descripcion']].groupby(['descripcion']).count()
ss_epb = incendios_cs[incendios_cs.epa == False][['idpif', 'descripcion']].groupby(['descripcion']).count()
ss = ss.merge(right=ss_epa, left_index=True, right_index=True, how='left')
ss = ss.merge(right=ss_epb, left_index=True, right_index=True, how='left')
ss.columns = ['Total', 'EPA', 'EPB']
ss = ss[['EPA', 'EPB', 'Total']]
ss = ss.fillna(0)
ss = ss.convert_dtypes()
ss.index.name = 'Situación sinóptica'

# CAUSAS

causas = pd.read_sql('''
            with c as 
                (select idcausa / 100 as ppal, count(*) as num, 
                sum(superficiearboladatotal) as sup_arbolada, 
                sum(superficinoarboladatotal) as sup_no_arbolada 
                from incendios_proximos_ze 
                group by idcausa 
                order by idcausa) 
            select 
                ppal, 
                sum(num) as num_incendios, 
                sum(sup_arbolada) as sup_arbolada, 
                sum(sup_no_arbolada) as sup_no_arbolada,
                sum(sup_arbolada) + sum(sup_no_arbolada) as sup_total
            from 
                c
            group by ppal 
            order by ppal
            ''', conn)

causas_detalle = pd.read_sql('''
            with c as 
                (select idcausa, count(*) as num, 
                sum(superficiearboladatotal) as sup_arbolada, 
                sum(superficinoarboladatotal) as sup_no_arbolada 
                from incendios_proximos_ze 
                group by idcausa 
                order by idcausa) 
            select 
                idcausa,
                descripcion_corta,
                sum(num) as num_incendios, 
                sum(sup_arbolada) as sup_arbolada, 
                sum(sup_no_arbolada) as sup_no_arbolada,
                sum(sup_arbolada) + sum(sup_no_arbolada) as sup_total
            from 
                c,
                causas 
            where 
                c.idcausa = causas.id
                and causas.ididioma = 0
            group by 
                descripcion_corta,
                idcausa
            order by c.idcausa
            ''', conn)
    
causas.index = ['Rayo', 'Negligencia', 'Accidente', 'Intencionado', 'Desconocida', 'Reproducción']
causas = causas[['num_incendios', 'sup_arbolada', 'sup_no_arbolada', 'sup_total']]

# SEVERIDAD 

incendios = [{'zona': 'Jerte-Tornavacas', 'idpif': ['1133522', '1143015', '1143273', '574176', '1164099']}, 
             {'zona': 'Villanueva-Madrigal', 'idpif': ['1122656', '1132420', '1133464', '1163320', '1163391']}, 
             {'zona': 'Losar', 'idpif': ['1163407', '569396', '1169767', '574250', '1122672']}]

severidad = []

zonas_dict = {'jerte': 0, 'tornavacas': 0, 'madrigal': 1, 'losar': 2}

severidad = incendios[zonas_dict[zona]]
severidad['incendios'] = []

for n in severidad['idpif']:
    perimetro = gpd.read_postgis("select * from perimetros_utm where idpif = {}".format(n), conn)
    severidad['incendios'].append({'fecha': perimetro.iloc[0]['fecha'].strftime('%d/%m/%Y'), 'idpif': n})

# CATASTRO 

if zona == 'madrigal':
    sql = '''
        SELECT nombre, masa, st_union(geom) as geom, sum (area) / 10000 as sup
        FROM catastro_mod_{}
        GROUP by nombre, masa
        '''.format(zona)
    sql += ''' union all
            select nombre, masa, st_union(geom) as geom, sum(area) / 10000 as sup
            from catastro_mod_villanueva
            GROUP by nombre, masa
            '''
    sql += ''' union all
            select nombre, masa, st_union(geom) as geom, sum(area) / 10000 as sup
            from catastro_mod_valverde
            GROUP by nombre, masa
            '''
elif zona == 'losar':
    sql = 'select nombre, st_union(geom) as geom, sum(area) / 10000 as sup from catastro_mod_{} group by nombre'. format(zona)
else:
    sql = 'select nombre, masa, st_union(geom) as geom, sum(area) / 10000 as sup from catastro_mod_{} group by nombre, masa'. format(zona)

parcelas = gpd.read_postgis(sql, conn)
catastro = parcelas.to_dict(orient='records')
display(catastro)

# FORESTALES
    
forestales = pd.read_sql("select forestal, contacto from forestales where zona = '{}'".format(zona), conn)

# HIDROGRAFÍA

rios = gpd.read_postgis('''select 
                row_number() over (order by a.long_km desc) as id_mapa, a.* 
                from rios a, zona_estudio_{} b 
                where st_intersects(a.geom, b.geom)
                and a.long_km < 50'''.format(zona), conn)

# METEO (TODO crear tabla en base de datos para no tener tanto código aquí)
datos_meteo = []
if comarca == 'vera':
    estaciones = ['madrigal', 'piornal']
else:
    estaciones = ['tornavacas', 'piornal']

for estacion in estaciones:
    dias_lluvia = pd.read_sql('''
                            select estacion, count(*) as dias_lluvia
                            from meteo_{} 
                            where prectotal > 0 and estacion is not null
                            and estacion != 'invierno_2020_2021'
                            group by estacion 
                            order by estacion'''.format(estacion), conn)
    dias_sin_lluvia_5 = pd.read_sql('''
                            select estacion, count(*) as dsl5
                            from meteo_{} 
                            where dias_sin_lluvia_5 is true and estacion is not null
                            group by estacion 
                            order by estacion'''.format(estacion), conn)
    dias_sin_lluvia_8 = pd.read_sql('''
                            select estacion, count(*) as dsl8
                            from meteo_{} 
                            where dias_sin_lluvia_8 is true and estacion is not null
                            group by estacion 
                            order by estacion'''.format(estacion), conn)
    nieve = pd.read_sql('''
                            select estacion, count(*) as nieve
                            from meteo_{}
                            where prectotal > 0 and tmin < 0 and estacion is not null
                            group by estacion 
                            order by estacion'''.format(estacion), conn)
    heladas = pd.read_sql('''
                            select estacion, count(*) as heladas
                            from meteo_{}
                            where tmin < 0 and estacion is not null
                            group by estacion 
                            order by estacion'''.format(estacion), conn)
    ventana_solana = pd.read_sql('''
                            select estacion, count(*) as vs
                            from meteo_{} 
                            where tmax < 21 and tmin > 6 and dias_sin_lluvia_5 is true and estacion is not null
                            group by estacion 
                            order by estacion'''.format(estacion), conn)
    ventana_umbria = pd.read_sql('''
                            select estacion, count(*) as vu
                            from meteo_{}
                            where tmax < 21 and tmin > 6 and dias_sin_lluvia_8 is true and estacion is not null
                            group by estacion 
                            order by estacion'''.format(estacion), conn)

    meteo = pd.merge(dias_lluvia, dias_sin_lluvia_5, how='left')
    meteo = pd.merge(meteo, dias_sin_lluvia_8, how='left')
    meteo = pd.merge(meteo, nieve, how='left')
    meteo = pd.merge(meteo, heladas, how='left')
    meteo = pd.merge(meteo, ventana_solana, how='left')
    meteo = pd.merge(meteo, ventana_umbria, how='left')
    meteo = meteo.set_index('estacion')
    meteo = meteo.fillna(0)
    meteo = meteo.convert_dtypes()
    meteo.loc['media'] = meteo.mean(axis=0)
    meteo.index.name = 'Periodo'
    meteo.columns=['días lluvia', '5 días sin lluvia', '8 días sin lluvia', 'nieve', 'heladas', 'ventana solana', 'ventana umbría']
    datos_meteo.append({'estacion': estacion, 'meteo':meteo})

# VEGETACIÓN

habitats = pd.read_sql('''
                        select distinct codue, nom_comun, generico, nom_habita, alianza, spsalianza, prioritari 
                        from atlashabitats2005_extremadura a,
                        zona_estudio_{} b
                        where st_intersects(st_transform(a.geom, 25830), b.geom)
                        '''.format(zona), conn)
habitats_prioritarios = habitats[habitats.prioritari == '*']
habitats = habitats[habitats.prioritari == 'Np']
habitats_anexo = habitats.to_dict(orient='records')

# ÁRBOLES SINGULARES

arboles_singulares = gpd.read_postgis('''
                    select a.* from arboles_singulares a,
                    zona_estudio_{} b
                    where st_intersects(st_transform(a.geom, 25830), b.geom)
                    '''.format(zona), conn).to_dict(orient='records')
    
# CERRAR CONEXIÓN BASE DE DATOS

conn.close()

# EXPORTAR 

capitulos = ['introduccion', 'objetivos', 'ambito', 'requisitos', 'puestos', 'planificacion', 'elementos', 'juicio']

env = Environment(loader=FileSystemLoader('static/templates_vera'))
template =env.get_template('base.html')
output = template.render(
    comarca = comarca,
    zona = zona,
    severidad = severidad,
    condiciones_sinopticas = incendios_cs.to_dict(orient='records'),
    ss = ss,
    catastro = catastro,
    forestales = forestales,
    capitulos = capitulos,
    rios = rios,
    datos_meteo = datos_meteo,
    causas = causas,
    causas_detalle = causas_detalle,
    habitats = habitats,
    habitats_prioritarios = habitats_prioritarios,
    habitats_anexo = habitats_anexo,
    arboles_singulares = arboles_singulares
)

with open('static/html/final.html', 'w') as f:
    f.write(output)

[{'nombre': 'mup',
  'geom': <shapely.geometry.multipolygon.MultiPolygon at 0x7f8fdcee81d0>,
  'sup': 4417.2867}]